In [ ]:
!git clone https://github.com/openai/shap-e.git
%cd shap-e
!pip install -e .


fatal: destination path 'shap-e' already exists and is not an empty directory.
/content/shap-e
Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-b3pgb324/clip_a10acbb37c3e48f8911a6a84b0522c01
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-b3pgb324/clip_a10acbb37c3e48f8911a6a84b0522c01
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Attempting uninstall: shap-e
    Found existing installation: shap-e 0.0.0
    Uninstalling shap-e-0.0.0:
      Successfully uninstalled shap-e-0.0.0
  Running setup.py develop for shap-e


In [ ]:

import torch
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, decode_latent_mesh,gif_widget
from shap_e.util.image_util import load_image
from datetime import datetime

ModuleNotFoundError: No module named 'onnxruntime'

In [ ]:
# Load models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xm = load_model('transmitter', device=device)
modelTxt = load_model('text300M', device=device)
modelImg = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

/content/shap-e/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/shap-e/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [ ]:
def generate_image_3d(image_p):
  batch_size = 4
  guidance_scale = 3.0

# To get the best result, you should remove the background and show only the object of interest to the model.
  image = load_image(image_p)

  latents = sample_latents(
    batch_size=batch_size,
    model=modelImg,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
    )
  render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
  size = 64 # this is the size of the renders; higher values take longer to render.
  cameras = create_pan_cameras(size, device)
  for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

In [ ]:
def generate_text_3d(prompt):
  batch_size = 3
  guidance_scale = 15.0
  prompt = prompt

  latents = sample_latents(
    batch_size=batch_size,
    model=modelTxt,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
    )
  render_mode = 'nerf' # you can change this to 'stf'
  size = 64
  cameras = create_pan_cameras(size, device)
  for i, latent in enumerate(latents):
      images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
      display(gif_widget(images))
# Example of saving the latents as meshes.
  from shap_e.util.notebooks import decode_latent_mesh

  for i, latent in enumerate(latents):
      t = decode_latent_mesh(xm, latent).tri_mesh()
      with open(f'/content/shapE_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
      with open(f'/content/ShapE_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)


In [ ]:
generate_text_3d("a sword")  # Text input


  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAALLA0LLAz7HA0LG/0LC/zrG+zrC+zq++zbC9za+9za+9y669y7…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAALK/x7G+xrG+xbC+x7C+xrC+xa++xq++xbC9xq+9xq+9xa69xq…

HTML(value='<img src="data:image/gif;base64,R0lGODlhgACAAIcAALPB0bLA0bK/0bG/0bG/0LC/0bC+0LC+z7C9z6++0K+9z669z6…

/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


In [ ]:
generate_image_3d("path/to/image.png")  # Image input

  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAADsAADrAADqAADpAADoAADnAADmAADlAADkAADjAADiAADhAA…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAADsAADrAADqAADpAADoAADnAADmAADlAADkAADjAADiAADhAA…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIYAAADqAADpAADoAADnAADmAADlAADkAADjAADiAADhAAHgAADgAA…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAADsAADrAADqAADpAADoAADnAADmAADlAADkAADjAADiAADhAA…